# 🛡️ Transaction Guardian - Interactive Demo

**CloudWalk Monitoring Intelligence Challenge - Task 3.2**

Este notebook demonstra o sistema de detecção de anomalias.

In [ ]:
!pip install -q pandas numpy scikit-learn plotly

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.ensemble import IsolationForest
from io import StringIO

print('✅ Setup completo!')

## 📊 Dados de Exemplo

In [ ]:
data = '''timestamp,status,count
2025-07-12 17:00:00,approved,104
2025-07-12 17:05:00,approved,113
2025-07-12 17:10:00,approved,64
2025-07-12 17:15:00,approved,85
2025-07-12 17:20:00,approved,96
2025-07-12 17:25:00,approved,79
2025-07-12 17:30:00,approved,124
2025-07-12 17:35:00,approved,135'''

df = pd.read_csv(StringIO(data))
df['timestamp'] = pd.to_datetime(df['timestamp'])
print(f'📊 {len(df)} registros carregados')
df

## 🤖 Detector de Anomalias

In [ ]:
class AnomalyDetector:
    def __init__(self):
        self.model = IsolationForest(contamination=0.1, random_state=42)
        
    def detect(self, df):
        df = df.copy()
        df['rolling_mean'] = df['count'].rolling(3, min_periods=1).mean()
        df['rolling_std'] = df['count'].rolling(3, min_periods=1).std().fillna(1)
        df['zscore'] = (df['count'] - df['rolling_mean']) / df['rolling_std']
        
        X = df[['count']].values
        self.model.fit(X)
        scores = self.model.decision_function(X)
        df['ml_score'] = 1 / (1 + np.exp(scores))
        
        df['combined_score'] = 0.6 * df['ml_score'] + 0.4 * np.minimum(np.abs(df['zscore']) / 3, 1)
        df['is_anomaly'] = (df['count'] < 50) | (df['count'] < df['rolling_mean'] * 0.5) | (df['combined_score'] > 0.7)
        return df

detector = AnomalyDetector()
df_analyzed = detector.detect(df)
anomalies = df_analyzed[df_analyzed['is_anomaly']]
print(f'🚨 {len(anomalies)} anomalias detectadas!')
df_analyzed[['timestamp', 'count', 'rolling_mean', 'combined_score', 'is_anomaly']]

## 📈 Visualização

In [ ]:
fig = make_subplots(rows=2, cols=1, subplot_titles=['Volume', 'Score'])

fig.add_trace(go.Scatter(x=df_analyzed['timestamp'], y=df_analyzed['count'], name='Transações'), row=1, col=1)
fig.add_trace(go.Scatter(x=df_analyzed['timestamp'], y=df_analyzed['rolling_mean'], name='Média', line=dict(dash='dash')), row=1, col=1)
fig.add_trace(go.Scatter(x=df_analyzed['timestamp'], y=df_analyzed['combined_score'], name='Score', fill='tozeroy'), row=2, col=1)
fig.add_hline(y=0.7, line_dash='dash', line_color='red', row=2, col=1)

fig.update_layout(title='🛡️ Transaction Guardian', height=500)
fig.show()

---
**Sérgio** | CloudWalk Challenge - Task 3.2 | January 2025